In [2]:
# Establish AWS session
import boto3
import sagemaker

from sagemaker import get_execution_role

session = boto3.Session()

role = get_execution_role()

In [3]:
# Import sample dataset

from sklearn import datasets

iris = datasets.load_iris()

In [4]:
# Split data and target

X = iris.data
y = iris.target

In [5]:
# Notice the data types
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [6]:
# Convert numpy arrays to float32 type, as required by the algorithm
X = X.astype("float32")
y = y.astype("float32")

X[:10,:], y[:10]

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [43]:
# Convert data to record-IO wrapped protobuf: Sagemaker algorithms generally require this
import io
import numpy as np
import sagemaker.amazon.common as smac


buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, X, y)
buf.seek(0)

0

In [44]:
# Now it has been converted to the right format
buf

In [45]:
# Upload data to S3: Sagemaker reads record-IO wrapped protobuf from S3 for model training
bucket = sagemaker.Session().default_bucket()

boto3.resource("s3").Bucket(bucket).Object("iris-linear-learner/train-pb").upload_fileobj(buf)

train_data = f"s3://{bucket}/iris-linear-learner/train-pb"

In [47]:
# Set s3 location for saved model artifacts
artifact_location = f"s3://{bucket}/iris-linear-learner/artfacts"

In [48]:
# Retrieve the container image containing the Sagemaker-provided model
from sagemaker.image_uris import retrieve

container = retrieve("linear-learner", session.region_name)

In [ ]:
# Train the model (not including model evaluation)
sess = sagemaker.Session()

model = sagemaker.estimator.Estimator(
    container, # container image for model
    role, # execution role
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=artifact_location,
    sagemaker_session=sess
)
model.set_hyperparameters(feature_dim='auto', predictor_type="multiclass_classifier", num_classes=3, mini_batch_size=10)

model.fit({"train": train_data})

In [60]:
# Deploy model API endpoint
endpoint_name = f"iris-linearlearner-demo"

model_endpoint = model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge", endpoint_name=endpoint_name)

-----------!